<a href="https://colab.research.google.com/github/talkin24/jaxflax_lab/blob/main/Parallel_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensembling on multiple devices

앙상블의 크기가 사용 가능한 디바이스 수와 동일한 MNIST 데이터 세트에서 CNN의 앙상블을 훈련하는 방법을 보여드리겠습니다. 간단히 설명하면 다음과 같습니다:

- `jax.pmap()`을 사용하여 여러 함수를 병렬로 만듭니다,

- 랜덤 시드를 분할하여 다른 매개변수 초기화를 얻습니다,

- 입력을 복제하고 필요한 경우 출력을 복제 해제합니다,

- 예측을 계산하기 위해 여러 기기에서 평균 확률을 계산합니다.

이 하우투에서는 임포트, CNN 모듈, 메트릭 계산과 같은 일부 코드를 생략했지만 이러한 코드는 MNIST 예제에서 찾을 수 있습니다.

## Parallel functions

먼저 모델의 초기 파라미터를 검색하는 `create_train_state()`의 병렬 버전을 생성합니다. 이 작업은 `jax.pmap()`을 사용하여 수행합니다. 함수를 "pmap"하는 효과는 함수를 XLA로 컴파일하지만(jax.jit()와 유사), XLA 디바이스(예: GPU/TPU)에서 병렬로 실행하는 것입니다.

In [ ]:
# Single-model
def create_train_state(rng, learning_rate, momentum):
  cnn = CNN()
  params = cnn.init(rng, jnp.ones([1, 28, 28, 1]))['params']
  tx = optax.sgd(learning_rate, momentum)
  return train_state.TrainState.create(
      apply_fn=cnn.apply, params=params, tx=tx)
  

# Ensemble
@functools.partial(jax.pmap, static_broadcasted_argnums=(1, 2)) #####
def create_train_state(rng, learning_rate, momentum):
  cnn = CNN()
  params = cnn.init(rng, jnp.ones([1, 28, 28, 1]))['params']
  tx = optax.sgd(learning_rate, momentum)
  return train_state.TrainState.create(
      apply_fn=cnn.apply, params=params, tx=tx)

위의 단일 모델 코드의 경우 `jax.jit()`를 사용하여 모델을 느리게 초기화합니다. 

앙상블의 경우 `jax.pmap()`은 기본적으로 제공된 인수 `rng`의 첫 번째 축에 매핑되므로 나중에 이 함수를 호출할 때 각 디바이스에 대해 다른 값을 제공해야 합니다.

또한 `learning_rate`와 `momentum`을 정적 인자로 지정하여 추상적인 모양이 아닌 이 인자의 구체적인 값을 사용하도록 지정한 점에 유의하세요. 이는 제공된 인수가 스칼라 값이기 때문에 필요합니다. 

다음으로 `apply_model()` 및 `update_model()` 함수에 대해서도 동일한 작업을 수행합니다. 

앙상블에서 예측을 계산하기 위해 개별 확률의 평균을 취합니다. `jax.lax.pmean()`을 사용하여 여러 기기에서 평균을 계산합니다. 이를 위해서는 `jax.pmap()` 및 `jax.lax.pmean()` 모두에 axis_name을 지정해야 합니다.

In [ ]:
#Single-model
@jax.jit #####
def apply_model(state, images, labels):
  def loss_fn(params):
    logits = CNN().apply({'params': params}, images)
    one_hot = jax.nn.one_hot(labels, 10)
    loss = optax.softmax_cross_entropy(logits=logits, labels=one_hot).mean()
    return loss, logits

  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (loss, logits), grads = grad_fn(state.params)

  accuracy = jnp.mean(jnp.argmax(logits, -1) == labels) #####
  return grads, loss, accuracy

@jax.jit #####
def update_model(state, grads):
  return state.apply_gradients(grads=grads)


# Ensemble
@functools.partial(jax.pmap, axis_name='ensemble') #####
def apply_model(state, images, labels):
  def loss_fn(params):
    logits = CNN().apply({'params': params}, images)
    one_hot = jax.nn.one_hot(labels, 10)
    loss = optax.softmax_cross_entropy(logits=logits, labels=one_hot).mean()
    return loss, logits

  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (loss, logits), grads = grad_fn(state.params)
  probs = jax.lax.pmean(jax.nn.softmax(logits), axis_name='ensemble') #####
  accuracy = jnp.mean(jnp.argmax(probs, -1) == labels) #####
  return grads, loss, accuracy

@jax.pmap #####
def update_model(state, grads):
  return state.apply_gradients(grads=grads)

## Training the Ensemble

다음으로 `train_epoch()` 함수를 변환합니다.

위에서 pmapped 함수를 호출할 때 필요한 경우 모든 디바이스에 대한 인수를 복제하고 반환값의 중복을 제거하는 작업을 주로 처리해야 합니다.

In [ ]:
# Single-model
def train_epoch(state, train_ds, batch_size, rng):
  train_ds_size = len(train_ds['image'])
  steps_per_epoch = train_ds_size // batch_size

  perms = jax.random.permutation(rng, len(train_ds['image']))
  perms = perms[:steps_per_epoch * batch_size]
  perms = perms.reshape((steps_per_epoch, batch_size))

  epoch_loss = []
  epoch_accuracy = []

  for perm in perms:
    batch_images = train_ds['image'][perm, ...] #####
    batch_labels = train_ds['label'][perm, ...] #####
    grads, loss, accuracy = apply_model(state, batch_images, batch_labels)
    state = update_model(state, grads)
    epoch_loss.append(loss) #####
    epoch_accuracy.append(accuracy) #####
  train_loss = np.mean(epoch_loss)
  train_accuracy = np.mean(epoch_accuracy)
  return state, train_loss, train_accuracy


# Ensemble
def train_epoch(state, train_ds, batch_size, rng):
  train_ds_size = len(train_ds['image'])
  steps_per_epoch = train_ds_size // batch_size

  perms = jax.random.permutation(rng, len(train_ds['image']))
  perms = perms[:steps_per_epoch * batch_size]
  perms = perms.reshape((steps_per_epoch, batch_size))

  epoch_loss = []
  epoch_accuracy = []

  for perm in perms:
    batch_images = jax_utils.replicate(train_ds['image'][perm, ...]) #####
    batch_labels = jax_utils.replicate(train_ds['label'][perm, ...]) #####
    grads, loss, accuracy = apply_model(state, batch_images, batch_labels)
    state = update_model(state, grads)
    epoch_loss.append(jax_utils.unreplicate(loss)) #####
    epoch_accuracy.append(jax_utils.unreplicate(accuracy)) #####
  train_loss = np.mean(epoch_loss)
  train_accuracy = np.mean(epoch_accuracy)
  return state, train_loss, train_accuracy

보시다시피 `state`와 관련된 로직을 변경할 필요가 없습니다. 

아래 훈련 코드에서 볼 수 있듯이 train state는 이미 복제되어 있으므로 `train_step()`에 전달하면 `train_step()`이 pmap되어 있기 때문에 정상적으로 작동하기 때문입니다. 그러나 훈련 데이터 세트는 아직 복제되지 않았으므로 여기서 리플리케이션을 수행합니다. 전체 train 데이터 집합을 복제하는 것은 너무 많은 메모리를 사용하므로 배치 수준에서 수행합니다.

이제 실제 훈련 로직을 다시 작성할 수 있습니다. 이는 두 가지 간단한 변경 사항으로 구성됩니다. RNG를 `create_train_state()`에 전달할 때 복제되는지 확인하는 것과 전체 데이터 세트에 대해 직접 이 작업을 수행할 수 있도록 훈련 데이터 세트보다 훨씬 작은 테스트 데이터 세트를 리플리케이트하는 것입니다.

In [ ]:
# Single-model
train_ds, test_ds = get_datasets()

rng = jax.random.PRNGKey(0)

rng, init_rng = jax.random.split(rng)
state = create_train_state(init_rng, learning_rate, momentum) #####


for epoch in range(1, num_epochs + 1):
  rng, input_rng = jax.random.split(rng)
  state, train_loss, train_accuracy = train_epoch(
      state, train_ds, batch_size, input_rng)

  _, test_loss, test_accuracy = apply_model( #####
      state, test_ds['image'], test_ds['label']) #####

  logging.info(
      'epoch:% 3d, train_loss: %.4f, train_accuracy: %.2f, '
      'test_loss: %.4f, test_accuracy: %.2f'
      % (epoch, train_loss, train_accuracy * 100, test_loss,
         test_accuracy * 100))


# Ensemble
train_ds, test_ds = get_datasets()
test_ds = jax_utils.replicate(test_ds) #####
rng = jax.random.PRNGKey(0)

rng, init_rng = jax.random.split(rng)
state = create_train_state(jax.random.split(init_rng, jax.device_count()), #####
                           learning_rate, momentum) #####

for epoch in range(1, num_epochs + 1):
  rng, input_rng = jax.random.split(rng)
  state, train_loss, train_accuracy = train_epoch(
      state, train_ds, batch_size, input_rng)

  _, test_loss, test_accuracy = jax_utils.unreplicate( #####
      apply_model(state, test_ds['image'], test_ds['label'])) #####

  logging.info(
      'epoch:% 3d, train_loss: %.4f, train_accuracy: %.2f, '
      'test_loss: %.4f, test_accuracy: %.2f'
      % (epoch, train_loss, train_accuracy * 100, test_loss,
         test_accuracy * 100))

# Scale up Flax Modules on multiple devices with `pjit`

이 가이드에서는 JAX의 `pjit` 및 `flax.linen.spmd`를 사용하여 여러 장치 및 호스트에서 Flax 모듈을 확장하는 방법을 보여줍니다.

## Flax and `pjit`

`jxa.experimental.pjit`은 JAX 계산을 자동으로 컴파일하고 확장하는 방법을 제공합니다. `pjit`에는 다음과 같은 이점이 있습니다:

- `pjit`은 `jax.jit`과 유사한 인터페이스를 가지고 있으며 컴파일해야 하는 함수의 데코레이터로 작동합니다.

- `pjit`을 사용하면 단일 기기에서 실행되는 것처럼 코드를 작성할 수 있으며, 단일 프로그램 다중 데이터(SPMD) 패러다임을 사용하여 여러 기기에서 자동으로 컴파일 및 실행됩니다.

- `pjit`을 사용하면 코드의 입력과 출력을 여러 기기에서 분할하는 방법을 명시할 수 있으며 컴파일러가 그 방법을 알아서 처리합니다: 1) 내부의 모든 것을 분할하고, 2) 디바이스 간 통신을 컴파일합니다.


Flax는 `Flax Module`에서 `pjit`을 사용하는 데 도움이 되는 다음과 같은 몇 가지 기능을 제공합니다:

1. `flax.linen.Module`을 정의할 때 데이터의 파티션을 지정하는 인터페이스.

2. `pjit` 실행에 필요한 파티션 정보를 생성하는 유틸리티 함수.

3. "logical axis annotations"이라는 축 이름을 사용자 정의하는 인터페이스로, 모듈 코드와 파티션 계획을 분리하여 다양한 파티션 레이아웃을 더 쉽게 실험할 수 있습니다.

## Setup

In [1]:
# Once Flax v0.6.4 is released, use `pip3 install flax`.
! pip3 install -qq "git+https://github.com/google/flax.git@main#egg=flax"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


## Imports 

In [2]:
import os
os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=8'

In [3]:
import functools
import numpy as np
import jax

from jax import lax, random, numpy as jnp

import flax
from flax import struct, traverse_util, linen as nn
from flax.linen import spmd # Flax Linen SPMD.
from flax.core import freeze, unfreeze
from flax.training import train_state, checkpoints

import optax # Optax for common losses and optimizers. 

1. 2x4 디바이스 메시(8개 디바이스)를 시작합니다(TPU v3-8의 레이아웃과 동일).

2. 각 축에 이름을 붙입니다. 축 이름에 주석을 다는 일반적인 방법은 `('data', 'model')`이며, 여기서 '데이터'는 다음과 같습니다:

- `data`: 입력 및 활성화의 배치 차원에 대한 데이터 병렬 샤딩에 사용되는 메시 차원입니다.

- `model`: 여러 기기에서 모델의 매개변수를 샤딩하는 데 사용되는 메시 차원입니다.

In [4]:
from jax.experimental.pjit import pjit, with_sharding_constraint
from jax.sharding import Mesh, PartitionSpec
from jax.experimental import mesh_utils

# Start a device mesh.
device_mesh = mesh_utils.create_device_mesh((2, 4))
print(device_mesh)
# Annotate each axis with a name.
mesh = Mesh(devices=device_mesh, axis_names=('data', 'model'))
mesh

[[CpuDevice(id=0) CpuDevice(id=1) CpuDevice(id=2) CpuDevice(id=3)]
 [CpuDevice(id=4) CpuDevice(id=5) CpuDevice(id=6) CpuDevice(id=7)]]


Mesh(device_ids=array([[0, 1, 2, 3],
       [4, 5, 6, 7]]), axis_names=('data', 'model'))

## Define a layer

모델을 정의하기 전에 dot product를 위한 두 개의 매개변수 `W1`과 `W2`를 생성하고 그 사이에 `jax.nn.relu`(ReLU) 활성화 함수를 사용하는 `DotReluDot`이라는 예제 레이어를 생성합니다(`flax.linen.Module`을 서브클래스화 함으로써).

`pjit`에서 이 레이어를 효율적으로 사용하려면 다음 API를 적용하여 매개변수와 중간 변수에 올바르게 주석을 달아야 합니다:

1. 매개변수 `W1` 및 `W2`를 생성할 때 `flax.linen.with_partitioning`을 사용하여 이니셜라이저 함수를 decorate하세요.

2. 이상적인 제약 조건이 알려진 경우 `pjit.with_sharding_constraint`를 적용하여 `y` 및 `z`와 같은 중간 변수에 주석을 달아 특정 샤딩 패턴을 `pjit`에서 강제로 적용합니다.

- 이 단계는 선택 사항이지만 때때로 auto-SPMD가 효율적으로 파티셔닝하는 데 도움이 될 수 있습니다. 아래 예제에서는 이 호출이 필요하지 않은데, 이는 `pjit`이 `y`와 `z`에 대해 동일한 샤딩 레이아웃을 알아내기 때문입니다.

In [5]:
class DotReluDot(nn.Module):
  depth: int
  @nn.compact
  def __call__(self, x):
    W1 = self.param(
        'W1', 
        nn.with_partitioning(nn.initializers.xavier_normal(), (None, 'model')),
        (x.shape[-1], self.depth))

    y = jax.nn.relu(jnp.dot(x, W1))
    # Force a local sharding annotation.
    y = with_sharding_constraint(y, PartitionSpec('data', 'model'))

    W2 = self.param(
        'W2', 
        nn.with_partitioning(nn.initializers.xavier_normal(), ('model', None)),
        (self.depth, x.shape[-1]))

    z = jnp.dot(y, W2)
    # Force a local sharding annotation.
    z = with_sharding_constraint(z, PartitionSpec('data', None))

    # Return a tuple to conform with the API `flax.linen.scan` as shown in the cell below.
    return z, None

`'data'`, `'model'` 또는 `None`과 같은 디바이스 축 이름은 `flax.linen.with_partitioning`과 `pjit_with_sharding_constraint` API 호출에 모두 전달됩니다. 이는 이 데이터의 각 차원을 디바이스 메시 차원 중 하나에 걸쳐 샤딩하거나 전혀 샤딩하지 않는 방식을 나타냅니다.

예를 들어

- shape `(x.shape[-1], self.depth)`으로 `W1`을 정의하고 주석을 `(None, 'model')`로 지정하는 경우:

  - 첫 번째 차원(길이 `x.shape[-1]`)은 모든 기기에서 복제됩니다.

  - 두 번째 차원(길이 `self.depth`)은 디바이스 메시의 `model` 축에 걸쳐 샤딩됩니다. 즉, `W1`은 이 차원에서 `(0, 4)`, `(1, 5)`, `(2, 6)`, `(3, 7)` 장치에서 4방향으로 샤드됩니다.

- 출력 z에 `('data', None)`으로 주석을 달 때:

  - 첫 번째 차원인 배치 차원이 `'data'` 축에 걸쳐 샤드됩니다. 즉, 배치의 절반은 장치 `0-3`(처음 4개의 장치)에서 처리되고 나머지 절반은 장치 `4-7`(나머지 4개의 장치)에서 처리됩니다.

  - 두 번째 차원인 데이터 깊이 차원은 모든 디바이스에서 복제됩니다.

## Define a model with `flax.linen.scan` lifted transformation

이 가이드에서는 `flax.linen.scan`을 사용하여 `scan`과 같은 Flax lifed transforms이 JAX `pjit`과 함께 작동하는 방법을 보여줍니다.

`DotReluDot`을 생성한 후 `MLP` 모델을 (`flax.linen.Module`을 서브클래스화하여) `DotReluDot`의 여러 레이어로 정의합니다.

동일한 레이어를 복제하려면 `flax.linen.scan` 또는 for-loop를 사용할 수 있습니다:

- `flax.linen.scan`은 더 빠른 컴파일 시간을 제공할 수 있습니다.

- 런타임에는 for-loop가 더 빠를 수 있습니다.

아래 코드는 두 가지 방법을 모두 적용하는 방법을 보여줍니다.

참고: `flax.linen.scan`에는 매개변수에 대한 또 다른 차원(`scan`이 적용되는 차원)이 있습니다. 이 차원의 파티션에 주석을 달려면 `metadata_params` 인수를 사용해야 합니다. `DotReluDot`(a sub-`Module`) 내부의 매개변수는 이미 `model` 축을 따라 분할되어 있으므로 여기서는 모델 차원에 걸쳐 여러 레이어를 분할할 필요가 없으므로 `None`으로 표시해야 합니다.

In [6]:
class MLP(nn.Module):
  num_layers: int
  depth: int
  use_scan: bool
  @nn.compact
  def __call__(self, x):
    if self.use_scan:
      x, _ = nn.scan(DotReluDot, length=self.num_layers, 
                     variable_axes={"params": 0},
                     split_rngs={"params": True},
                     metadata_params={nn.PARTITION_NAME: None}
                     )(self.depth)(x)
    else:
      for i in range(self.num_layers):
        x, _ = DotReluDot(self.depth)(x)
    return x


## Specify sharding (includes initialization and `TrainState` creation)

다음으로, `pjit`이 입력 및 출력 데이터의 어노테이션으로 수신해야 하는 `jax.sharding.PartitionSpec`을 생성합니다. `PartitionSpec`은 2x4 메시에서 2축의 튜플입니다.

### Specify the input

데이터 병렬 처리의 경우, 배치 축을 `'data'`로 표시하여 `'data'` 축 전체에 배치 입력 `x`를 샤드할 수 있습니다:


In [7]:
x_spec = PartitionSpec('data', None)  # dimensions: (batch, length)
x_spec

PartitionSpec('data', None)

## Generate a `PartitionSpec` for the output

그런 다음, 출력에 대한 `PartitionSpec`을 생성하고 실제 출력을 참조로 사용해야 합니다.

1. 모델을 인스턴스화합니다.

2. `jax.eval_shape`를 사용하여 `model.init`을 추상적으로 평가합니다.

3. `flax.linen.get_partition_spec`을 사용하여 `PartitionSpec`을 자동으로 생성합니다.

아래 코드는 초기화 및 훈련 단계를 수행하기 위해 `flax.training.train_state`를 사용하는 경우 출력 사양을 얻는 방법을 보여 주며, 이 경우 `pjit`된 함수는 `TrainState`를 출력합니다.

(더 간단한 경우에는 `variables = model.init(k, x)`와 같이 변수 딕셔너리를 `pjit`된 함수의 출력으로 선택할 수도 있습니다. 이것도 작동합니다.)

In [8]:
# MLP hyperparameters.
BATCH, LAYERS, DEPTH, USE_SCAN = 8, 4, 1024, True
# Create fake inputs.
x = jnp.ones((BATCH, DEPTH))
# Initialize a PRNG key.
k = random.PRNGKey(0)

# Create an Optax optimizer.
optimizer = optax.adam(learning_rate=0.001)
# Instantiate the model.
model = MLP(LAYERS, DEPTH, USE_SCAN)

# A functional way of model initialization.
def init_fn(k, x, model, optimizer):
  variables = model.init(k, x) # Initialize the model.
  state = train_state.TrainState.create( # Create a `TrainState`.
    apply_fn=model.apply,
    params=variables['params'],
    tx=optimizer)
  return state

with mesh:
  # Create an abstract closure to wrap the function before feeding it in
  # because `jax.eval_shape` only takes pytrees as arguments`.
  abstract_variables = jax.eval_shape(
      functools.partial(init_fn, model=model, optimizer=optimizer), k, x)
# This `state_spec` has the same pytree structure as the output
# of the `init_fn`.
state_spec = nn.get_partition_spec(abstract_variables)
state_spec

TrainState(step=PartitionSpec(), apply_fn=<bound method Module.apply of MLP(
    # attributes
    num_layers = 4
    depth = 1024
    use_scan = True
)>, params=FrozenDict({
    ScanDotReluDot_0: {
        W1: PartitionSpec(None, None, 'model'),
        W2: PartitionSpec(None, 'model', None),
    },
}), tx=GradientTransformationExtraArgs(init=<function chain.<locals>.init_fn at 0x7f0c8c2831c0>, update=<function chain.<locals>.update_fn at 0x7f0c8c283370>), opt_state=(ScaleByAdamState(count=PartitionSpec(), mu=FrozenDict({
    ScanDotReluDot_0: {
        W1: PartitionSpec(None, None, 'model'),
        W2: PartitionSpec(None, 'model', None),
    },
}), nu=FrozenDict({
    ScanDotReluDot_0: {
        W1: PartitionSpec(None, None, 'model'),
        W2: PartitionSpec(None, 'model', None),
    },
})), EmptyState()))

## Apply `pjit` to complie the code

이제 `jax.jit`와 비슷한 방식으로 `init_fn`에 JAX `pjit`을 적용할 수 있지만, 두 개의 추가 인수인 `in_axis_resources`와 `out_axis_resources`를 추가할 수 있습니다.

`pjit`ted 함수를 실행할 때 `with mesh:` 컨텍스트를 추가해야 장치에 데이터를 올바르게 할당하기 위해 `mesh`(`jax.sharding.Mesh`의 인스턴스)를 참조할 수 있습니다.

In [9]:
pjit_init_fn = pjit(init_fn,
                    static_argnums=(2, 3),
                    in_axis_resources=(PartitionSpec(None), x_spec),  # PRNG key and x
                    out_axis_resources=state_spec
                    )
with mesh:
  initialized_state = pjit_init_fn(k, x, model, optimizer)
jax.tree_map(jnp.shape, initialized_state)

TrainState(step=(), apply_fn=<bound method Module.apply of MLP(
    # attributes
    num_layers = 4
    depth = 1024
    use_scan = True
)>, params=FrozenDict({
    ScanDotReluDot_0: {
        W1: Partitioned(value=(4, 1024, 1024), names=(None, None, 'model'), mesh=None),
        W2: Partitioned(value=(4, 1024, 1024), names=(None, 'model', None), mesh=None),
    },
}), tx=GradientTransformationExtraArgs(init=<function chain.<locals>.init_fn at 0x7f0c8c2831c0>, update=<function chain.<locals>.update_fn at 0x7f0c8c283370>), opt_state=(ScaleByAdamState(count=(), mu=FrozenDict({
    ScanDotReluDot_0: {
        W1: Partitioned(value=(4, 1024, 1024), names=(None, None, 'model'), mesh=None),
        W2: Partitioned(value=(4, 1024, 1024), names=(None, 'model', None), mesh=None),
    },
}), nu=FrozenDict({
    ScanDotReluDot_0: {
        W1: Partitioned(value=(4, 1024, 1024), names=(None, None, 'model'), mesh=None),
        W2: Partitioned(value=(4, 1024, 1024), names=(None, 'model', None), mes

## Inspect the Module output

`initialized_state`의 출력에서 `params` `W1`과 `W2`는 `flax.linen.Partitioned` 유형입니다. 이것은 Flax가 관련된 메타데이터를 기록할 수 있도록 실제 `jax.Array`를 감싸는 래퍼입니다. `.value`를 추가하거나 `.unbox()`를 실행하여 원시 `jax.Array`에 액세스할 수 있습니다.

또한 JAX 배열의 기본 `jax.sharding`을 확인하면 분할 방식에 대한 힌트를 얻을 수 있습니다.

In [10]:
print(type(initialized_state.params['ScanDotReluDot_0']['W1']))
print(type(initialized_state.params['ScanDotReluDot_0']['W1'].value))
print(initialized_state.params['ScanDotReluDot_0']['W1'].value.shape)

<class 'flax.core.meta.Partitioned'>
<class 'jaxlib.xla_extension.ArrayImpl'>
(4, 1024, 1024)


In [11]:
print(initialized_state.params['ScanDotReluDot_0']['W1'].value.sharding)

GSPMDSharding({devices=[1,1,4,2]0,4,1,5,2,6,3,7 last_tile_dim_replicate})


`jax.tree_map`을 사용하면 JAX 배열의 딕셔너리와 동일한 방식으로 박스형 매개변수 딕셔너리에 대해 대량 계산을 수행할 수 있습니다.

In [12]:
diff = jax.tree_map(
    lambda a, b: a - b, 
    initialized_state.params['ScanDotReluDot_0'], initialized_state.params['ScanDotReluDot_0'])
print(jax.tree_map(jnp.shape, diff))
diff_array = diff['W1'].unbox()
print(type(diff_array))
print(diff_array.shape)

FrozenDict({
    W1: Partitioned(value=(4, 1024, 1024), names=(None, None, 'model'), mesh=None),
    W2: Partitioned(value=(4, 1024, 1024), names=(None, 'model', None), mesh=None),
})
<class 'jaxlib.xla_extension.ArrayImpl'>
(4, 1024, 1024)


## Apply `pjit` to the train step and inference

이제 `pjit` 트레이닝 단계를 생성합니다

In [13]:
def train_step(state, x):
  # A fake loss function.
  def loss_unrolled(params):
    y = model.apply({'params': params}, x)
    return y.sum()
  grad_fn = jax.grad(loss_unrolled)
  grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  return state

pjit_step_fn = pjit(train_step,
                    in_axis_resources=(state_spec, x_spec),  # input annotations
                    out_axis_resources=state_spec,           # output annotations
                    )
with mesh:
  new_state = pjit_step_fn(initialized_state, x)

추론에 `pjit`을 적용합니다. `jax.jit`와 유사하게 `@functools.partial(pjit, ...)`과 같은 데코레이터를 사용하여 함수를 직접 컴파일할 수 있습니다.

In [14]:
@functools.partial(pjit, in_axis_resources=(state_spec, x_spec), out_axis_resources=x_spec)
def pjit_apply_fn(state, x):
  return state.apply_fn({'params': state.params}, x)

with mesh:
  y = pjit_apply_fn(new_state, x)
print(type(y))
print(y.dtype)
print(y.shape)

<class 'jaxlib.xla_extension.ArrayImpl'>
float32
(8, 1024)


## Profiling

TPU 팟 또는 팟 슬라이스에서 실행 중인 경우, 아래에 정의된 대로 사용자 정의 block_all 유틸리티 함수를 사용하여 성능을 측정할 수 있습니다

In [15]:
%%timeit

def block_all(xs):
  jax.tree_map(lambda x: x.block_until_ready(), xs)
  return xs

with mesh:
  new_state = block_all(pjit_step_fn(initialized_state, x))


621 ms ± 89.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Logical axis annotation

JAX auto SPMD는 사용자가 최적의 샤딩 레이아웃을 찾기 위해 다양한 샤딩 레이아웃을 탐색하도록 권장합니다. 이를 위해 Flax에서는 실제로 `'data'`, `'model'`과 같은 장치 메시 축 이름뿐만 아니라 더 설명적인 축 이름으로 모든 데이터의 차원에 주석을 달 수 있습니다.

아래의 `LogicalDotReluDot` 및 `LogicalMLP` 모듈 정의는 다음을 제외하고는 앞서 생성한 모듈과 유사합니다:

1. 모든 축에는 `'embed'`, `'hidden'`, `'batch'` 및 `'layer'`와 같은 보다 구체적이고 의미 있는 이름이 주석으로 지정되어 있습니다. 이러한 이름을 Flax에서는 논리적 축 이름이라고 합니다. 이러한 이름을 사용하면 모델 정의 내부의 차원 변경 사항을 더 읽기 쉽게 만들 수 있습니다.

2. `flax.linen.spmd.with_logical_partitioning`은 `flax.linen.with_partitioning`을 대체하고, `flax.linen.spmd.with_logical_constraint`는 `pjit.with_sharding_constraint`를 대체하여 논리적 축 이름을 인식할 수 있습니다.

In [16]:
class LogicalDotReluDot(nn.Module):
  depth: int
  @nn.compact
  def __call__(self, x):
    W1 = self.param(
        'W1', 
        spmd.with_logical_partitioning(nn.initializers.xavier_normal(), ('embed', 'hidden')),
        (x.shape[-1], self.depth)) 

    y = jax.nn.relu(jnp.dot(x, W1))
    # Force a local sharding annotation.
    y = spmd.with_logical_constraint(y, ('batch', 'hidden'))

    W2 = self.param(
        'W2', 
        spmd.with_logical_partitioning(nn.initializers.xavier_normal(), ('hidden', 'embed')),
        (self.depth, x.shape[-1]))

    z = jnp.dot(y, W2)
    # Force a local sharding annotation.
    z = spmd.with_logical_constraint(z, ('batch', 'embed'))
    return z, None

class LogicalMLP(nn.Module):
  num_layers: int
  depth: int
  use_scan: bool
  @nn.compact
  def __call__(self, x):
    if self.use_scan:
      x, _ = nn.scan(LogicalDotReluDot, length=self.num_layers, 
                    variable_axes={"params": 0},
                    split_rngs={"params": True},
                    metadata_params={nn.PARTITION_NAME: 'layer'}
                    )(self.depth)(x)
    else:
      for i in range(self.num_layers):
        x, _ = DotReluDot(self.depth)(x)
    return x

`LogicalMLP` 모델 정의는 논리적 축 이름을 가진 `PartitionSpec` 집합을 생성합니다.

이전 단계를 반복합니다. 모델을 인스턴스화하고, `init_fn`을 추상적으로 평가한 다음, `flax.linen.get_partition_spec`을 사용하여 `PartitionSpec`을 자동으로 생성합니다:

In [17]:
logical_model = LogicalMLP(LAYERS, DEPTH, USE_SCAN)
logical_abstract_variables = jax.eval_shape(
    functools.partial(init_fn, model=logical_model, optimizer=optimizer), k, x)
logical_output_spec = nn.get_partition_spec(logical_abstract_variables)
logical_output_spec

TrainState(step=PartitionSpec(), apply_fn=<bound method Module.apply of LogicalMLP(
    # attributes
    num_layers = 4
    depth = 1024
    use_scan = True
)>, params=FrozenDict({
    ScanLogicalDotReluDot_0: {
        W1: PartitionSpec('layer', 'embed', 'hidden'),
        W2: PartitionSpec('layer', 'hidden', 'embed'),
    },
}), tx=GradientTransformationExtraArgs(init=<function chain.<locals>.init_fn at 0x7f0c8c2831c0>, update=<function chain.<locals>.update_fn at 0x7f0c8c283370>), opt_state=(ScaleByAdamState(count=PartitionSpec(), mu=FrozenDict({
    ScanLogicalDotReluDot_0: {
        W1: PartitionSpec('layer', 'embed', 'hidden'),
        W2: PartitionSpec('layer', 'hidden', 'embed'),
    },
}), nu=FrozenDict({
    ScanLogicalDotReluDot_0: {
        W1: PartitionSpec('layer', 'embed', 'hidden'),
        W2: PartitionSpec('layer', 'hidden', 'embed'),
    },
})), EmptyState()))

장치 메시가 모델을 올바르게 가져올 수 있도록 하려면 이러한 논리적 축 이름 중 장치 축 `'data'` 또는 `'model'`에 매핑되는 이름을 결정해야 합니다. 

이 규칙은 (`logical_axis_name`, `device_axis_name`) 튜플의 목록이며, `jax.linen.spmd.logical_to_mesh`는 이를 `pjit`가 허용하는 사양으로 변환합니다.

이를 통해 모델 정의를 수정하지 않고도 규칙을 변경하고 새로운 파티션 레이아웃을 시도해 볼 수 있습니다.

In [18]:
# Unspecified rule means unsharded by default, so no need to specify `('embed', None)` and `('layer', None)`.
rules = (('batch', 'data'),
         ('hidden', 'model'))

logical_state_spec = spmd.logical_to_mesh(logical_output_spec, rules)
logical_state_spec

TrainState(step=PartitionSpec(), apply_fn=<bound method Module.apply of LogicalMLP(
    # attributes
    num_layers = 4
    depth = 1024
    use_scan = True
)>, params=FrozenDict({
    ScanLogicalDotReluDot_0: {
        W1: PartitionSpec(None, None, 'model'),
        W2: PartitionSpec(None, 'model', None),
    },
}), tx=GradientTransformationExtraArgs(init=<function chain.<locals>.init_fn at 0x7f0c8c2831c0>, update=<function chain.<locals>.update_fn at 0x7f0c8c283370>), opt_state=(ScaleByAdamState(count=PartitionSpec(), mu=FrozenDict({
    ScanLogicalDotReluDot_0: {
        W1: PartitionSpec(None, None, 'model'),
        W2: PartitionSpec(None, 'model', None),
    },
}), nu=FrozenDict({
    ScanLogicalDotReluDot_0: {
        W1: PartitionSpec(None, None, 'model'),
        W2: PartitionSpec(None, 'model', None),
    },
})), EmptyState()))

여기의 `logical_state_spec`이 이전("non-logical") 예제의 `state_spec`과 동일한 내용을 가지고 있는지 확인할 수 있습니다. 이것은 분할된 함수를 생성할 때 지정한 `out_axis_resources`가 됩니다.

In [20]:
state_spec.params['ScanDotReluDot_0'] == logical_state_spec.params['ScanLogicalDotReluDot_0']

True

In [21]:
logical_pjit_init_fn = pjit(init_fn,
                            static_argnums=(2, 3),
                            in_axis_resources=(PartitionSpec(None), x_spec),  # RNG key and x
                            out_axis_resources=logical_state_spec
                            )
with mesh:
  logical_initialized_state = logical_pjit_init_fn(k, x, logical_model, optimizer)
jax.tree_map(jnp.shape, logical_initialized_state)

TrainState(step=(), apply_fn=<bound method Module.apply of LogicalMLP(
    # attributes
    num_layers = 4
    depth = 1024
    use_scan = True
)>, params=FrozenDict({
    ScanLogicalDotReluDot_0: {
        W1: LogicallyPartitioned(value=(4, 1024, 1024), names=('layer', 'embed', 'hidden'), mesh=None, rules=None),
        W2: LogicallyPartitioned(value=(4, 1024, 1024), names=('layer', 'hidden', 'embed'), mesh=None, rules=None),
    },
}), tx=GradientTransformationExtraArgs(init=<function chain.<locals>.init_fn at 0x7f0c8c2831c0>, update=<function chain.<locals>.update_fn at 0x7f0c8c283370>), opt_state=(ScaleByAdamState(count=(), mu=FrozenDict({
    ScanLogicalDotReluDot_0: {
        W1: LogicallyPartitioned(value=(4, 1024, 1024), names=('layer', 'embed', 'hidden'), mesh=None, rules=None),
        W2: LogicallyPartitioned(value=(4, 1024, 1024), names=('layer', 'hidden', 'embed'), mesh=None, rules=None),
    },
}), nu=FrozenDict({
    ScanLogicalDotReluDot_0: {
        W1: LogicallyPartit

## When to use device axis / logical axis

디바이스 또는 논리적 축을 사용할 시기는 모델의 파티셔닝을 얼마나 제어할 것인지에 따라 달라집니다.

매우 단순한 모델을 원하거나 파티셔닝 방식에 자신이 있는 경우, 디바이스 메시 축으로 정의하면 논리적 네이밍을 디바이스 네이밍으로 다시 변환하는 몇 줄의 추가 코드를 절약할 수 있습니다.

반면에 논리적 이름 지정 도우미는 다양한 샤딩 레이아웃을 탐색하는 데 유용합니다. 여러 가지를 실험해보고 모델에 가장 적합한 파티션 레이아웃을 찾으려는 경우 이 기능을 사용하세요.

정말 고급 사용 사례에서는 활성화 차원 이름에 매개변수 차원 이름과 다르게 주석을 달아야 하는 더 복잡한 샤딩 패턴이 있을 수 있습니다. 수동 메시 할당을 보다 세밀하게 제어하려는 경우 디바이스 축 이름을 직접 사용하는 것이 더 유용할 수 있습니다.

## Save the data

Save and load checkpoints guide - Multi-host/multi-process checkpointing에 표시된 대로 flax.training.checkpoints를 사용하여 교차 장치 배열을 저장할 수 있습니다. 이는 다중 호스트 환경(예: TPU 팟)에서 실행하는 경우 특히 필요합니다.

어레이를 원하는 파티션으로 복원하려면 각 JAX 어레이에 대해 동일한 구조를 가지며 원하는 `PartitionSpec`이 있는 샘플 대상 파이트리를 제공해야 한다는 점에 유의하세요. 배열을 복원하는 데 사용하는 `PartitionSpec`은 배열을 저장하는 데 사용한 것과 반드시 동일할 필요는 없습니다.